In [1]:
from example.crypto_store import InMemCryptoStore
from eventsourcing.encryption import CryptoRepository
from eventsourcing.repositories import EventStoreRepository
from example.event_store import InMemEventStore
from example.user import User
from datetime import date
from example.guid import guid

### Initialize the in-memory encryption key store, eventstore, and repositories

In [2]:
crypto_store = InMemCryptoStore()
event_store = InMemEventStore()

CryptoRepository.crypto_store = crypto_store
user_repo = EventStoreRepository[User](event_store, User)

### Create a user and show that its attributes are clear

In [3]:
user = User(guid(), "Paul", "Boulanger", date(1997,2,18))

print(user.id, user.first_name, user.last_name, user.date_of_birth.isoformat())


4fb61688-bc31-4d82-8315-d8fd9a505c7a Paul Boulanger 1997-02-18


### Save the user in the eventstore

The eventstore and key store are both empty

In [4]:
print(event_store.current)
print(crypto_store.store)

{}
{}


Saving the user in the eventstore

In [5]:
await user_repo.save(user,0)


The events related to the user are saved in the eventstore and an encryption key linked to the user was created and stored in the encryption key store

In [6]:
print(event_store.current)
print(crypto_store.store)

{'user-4fb61688-bc31-4d82-8315-d8fd9a505c7a': [<example.event_store.EventDescriptor object at 0x0000024DEF263410>]}
{'4fb61688-bc31-4d82-8315-d8fd9a505c7a': b'E9BHKs4jJ4MHM85L-Eb_7XePUqUAP0UqoNeBE2nrrfI='}


### Showing the encryption

The personnal data (in this case: `first_name`, `last_name`, `month_of_birth` and `day_of_birth`) are encrypted in the eventstore

In [7]:
event_store.current[User.to_stream_id(user.id)][0].event_data

'{"id": "4fb61688-bc31-4d82-8315-d8fd9a505c7a", "first_name": "encrypted_gAAAAABmjznC0lhrjq5TRZNl7z2pGKoklnS-E9OaeMJ50h3Dhnd92BuQJPgOBDGYlEwQOmS6c0feJjd440qK8JJvEQOYrpSV7A==", "last_name": "encrypted_gAAAAABmjznCm2m-FkZ3FXlR27oSdALe_49rqIyU20ZeDba9nxe1E3SFhbmF4YztgIo0yp-KY3IKLTP8YVbrxHrs41eDxSLc6Q==", "year_of_birth": 1997, "month_of_birth": "encrypted_gAAAAABmjznCesc_CvMMynLYd9nn-lmJ_aojZaDq0HXoyff0Y1qxcaf6OG6bcKao0tje7P19TRwWgkvte-VNWsu8Lqr6GBUzYg==", "day_of_birth": "encrypted_gAAAAABmjznCcSJWCHr7Zil97tETIgOfvJ6MuAV2RUWDAOZGtJ9gFDEDQwCU7-vq8YqyFytHksBuMr3AS8gSSXt2Zi7tncKgjA=="}'

Since there is an encryption key, we can retrieve the personnal informations of the user in a decrypted state

In [8]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='4fb61688-bc31-4d82-8315-d8fd9a505c7a', first_name='Paul', last_name='Boulanger', year_of_birth=1997, month_of_birth=2, day_of_birth=18)]

We delete the user's encryption key

In [9]:
crypto_store.remove(user.id)

Now the personnal informations are still encrypted

In [10]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='4fb61688-bc31-4d82-8315-d8fd9a505c7a', first_name='encrypted_gAAAAABmjznC0lhrjq5TRZNl7z2pGKoklnS-E9OaeMJ50h3Dhnd92BuQJPgOBDGYlEwQOmS6c0feJjd440qK8JJvEQOYrpSV7A==', last_name='encrypted_gAAAAABmjznCm2m-FkZ3FXlR27oSdALe_49rqIyU20ZeDba9nxe1E3SFhbmF4YztgIo0yp-KY3IKLTP8YVbrxHrs41eDxSLc6Q==', year_of_birth=1997, month_of_birth='encrypted_gAAAAABmjznCesc_CvMMynLYd9nn-lmJ_aojZaDq0HXoyff0Y1qxcaf6OG6bcKao0tje7P19TRwWgkvte-VNWsu8Lqr6GBUzYg==', day_of_birth='encrypted_gAAAAABmjznCcSJWCHr7Zil97tETIgOfvJ6MuAV2RUWDAOZGtJ9gFDEDQwCU7-vq8YqyFytHksBuMr3AS8gSSXt2Zi7tncKgjA==')]

When we load our user from the eventstore, the personnal information are encrypted or replaced by fake information

In [11]:
new_user = User()
new_user.loads_from_history(events)
print(new_user.id, new_user.first_name, new_user.last_name, new_user.date_of_birth.isoformat())

4fb61688-bc31-4d82-8315-d8fd9a505c7a encrypted_gAAAAABmjznC0lhrjq5TRZNl7z2pGKoklnS-E9OaeMJ50h3Dhnd92BuQJPgOBDGYlEwQOmS6c0feJjd440qK8JJvEQOYrpSV7A== encrypted_gAAAAABmjznCm2m-FkZ3FXlR27oSdALe_49rqIyU20ZeDba9nxe1E3SFhbmF4YztgIo0yp-KY3IKLTP8YVbrxHrs41eDxSLc6Q== 1997-01-01
